# Assignment - to build Golden Data Set using Synthetic Data Generation with RAGAS  and using different retrievers and with RAG Evaluation with LangSmith

The following retrievers are evaluated with Semantic Chunking enabled / disabled options.

## Install dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### Import NLTK

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DadaV\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DadaV\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### Load LangChain API key

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

Create a unique project name

In [21]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]
project_name = os.environ["LANGCHAIN_PROJECT"] = f"Philippines AI Bills RAG - {unique_id}"
project_name

'Philippines AI Bills RAG - a09da624'

Loan OpenAI and Cohere API Keys

In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")


In [6]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [7]:
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader

path = "bills/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())
# generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [9]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node 9f190615-0b38-43eb-b56d-b24c72c3c92d does not have a summary. Skipping filtering.
Node d284653c-539f-4d4e-af27-7030ae2485fe does not have a summary. Skipping filtering.
Node 3eb53c9f-4d84-431b-a798-f9f7be90f85b does not have a summary. Skipping filtering.
Node 3b5deefd-c165-48e9-8c01-51a05ef12653 does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 20, relationships: 138)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("bills/ai_law.json")
bills_data_kg = KnowledgeGraph.load("bills/ai_law.json")
bills_data_kg

KnowledgeGraph(nodes: 20, relationships: 138)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=bills_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [14]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

Finally, we can use our `TestSetGenerator` to generate our testset!

In [16]:
testset = generator.generate(testset_size=20, query_distribution=query_distribution)
testset.to_pandas()
testset.to_jsonl("bills/golden_dataset.json")  # Save for reuse

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

LLMDidNotFinishException: The LLM generation was not completed. Please increase the max_tokens and try again.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [17]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node 17a60be7-a9e0-4748-bde3-1f7e7bfc618b does not have a summary. Skipping filtering.
Node 0b752b2d-35ff-4b46-96ff-e5186c66c015 does not have a summary. Skipping filtering.
Node 1afecbc1-b488-4993-86ad-c605289dac61 does not have a summary. Skipping filtering.
Node 975684ca-2704-4790-ab22-2c5a2771664a does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [18]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What AI regulation is discussed in the context?,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The context discusses a bill introduced in the...,single_hop_specifc_query_synthesizer
1,Whay is the ris of Artificial Superinteligence?,[AI presents enormous opportunities for the Ph...,The context mentions that the global scientifi...,single_hop_specifc_query_synthesizer
2,Who is PIA S. CAYETANO in the context of the A...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,PIA S. CAYETANO is the senator who introduced ...,single_hop_specifc_query_synthesizer
3,What is Artificial General Intelligence and ho...,"[1 \na) Promote innovation, technological adva...",The context states that the Act shall regulate...,single_hop_specifc_query_synthesizer
4,How does the regulation of AI in employment an...,[<1-hop>\n\n1 \nd) Take full responsibility fo...,The regulation of AI in employment and labor p...,multi_hop_abstract_query_synthesizer
5,How does the review of AI-related employment t...,[<1-hop>\n\n1 \niii) \nProof of employer engag...,The review of AI-related employment trends is ...,multi_hop_abstract_query_synthesizer
6,how law penalties for not register ai and need...,[<1-hop>\n\n1 \nSec. 12. Contents of AI Regist...,The context explains that failure to register ...,multi_hop_abstract_query_synthesizer
7,How does the proposed Philippine AI regulation...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,The proposed Philippine AI regulation emphasiz...,multi_hop_abstract_query_synthesizer
8,how AI Ethics Review Board and AI registration...,[<1-hop>\n\n1 \ndevelopment priorities. These ...,"The AI Ethics Review Board, created by the NAI...",multi_hop_specific_query_synthesizer
9,How does the potential rise of Artificial Supe...,[<1-hop>\n\n1\n2\n3\n4\n5\n6\n7\n8 \n9\n10\n11...,The potential rise of Artificial Superintellig...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [22]:
from langsmith import Client

client = Client()

dataset_name = "Philippines AI Bills " + unique_id

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Philippines AI Bills"
)

dataset_name

'Philippines AI Bills a09da624'

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [23]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [24]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

p_chunk_size = 1000 # 500 in previous try

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = p_chunk_size,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [26]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

For the prompt, we will make it switchable to use with empathy version or non-empathy standard version. 

In [39]:
from langchain.prompts import ChatPromptTemplate

empathy = True

if empathy:

    EMPATHY_RAG_PROMPT = """\
    Given a provided context and question, you must answer the question based only on context.

    If you cannot answer the question based on the context - you must say "I don't know".

    You must answer the question using empathy and kindness, and make sure the user feels heard.

    Context: {context}
    Question: {question}
    """

    empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

else:

    RAG_PROMPT = """\
    You are a helpful and informative assistant. Given a provided context and question, you must answer the question based only on context.

    If you cannot answer the question based on the context, or you are unsure, you must say "I don't know".

    Context: {context}
    Question: {question}
    """

    rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)



For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [28]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4.1-mini")
chat_model = ChatOpenAI(model="gpt-4.1-nano")

As usual, we will power our RAG application with Qdrant!

Here we have version with semantic chunking and another version with recursive text splitter

In [29]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

semantic_documents = semantic_chunker.split_documents(rag_documents[:20])

# semantic chunking OFF - use rag_documents, else semantic_documents

from langchain_community.vectorstores import Qdrant

text_vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG")

semantic_vectorstore = Qdrant.from_documents(
    documents=semantic_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG")

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [40]:
eval_llm = ChatOpenAI(model="gpt-4.1")

from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator(
    "qa", 
    config={
        "llm" : eval_llm
        },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["response"],      # student's answer (model output)
        "reference": example.outputs["answer"],   # gold / true answer
        "input": example.inputs["question"],         # the question
        }
    )

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
        },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["response"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
        }
    )

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
        },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["response"],
        # "reference": example.outputs["answer"], # to silence the warning as it is not normal for empathy to have reference
        "input": example.inputs["question"],
        }    
    )

In [ ]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

from langchain.retrievers.multi_query import MultiQueryRetriever

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models
from langchain_qdrant import QdrantVectorStore

from langchain.retrievers import EnsembleRetriever

# Test for a single retriever only
retriever_names = [
    "Naive"
]

# Enable for multi-retriever tests
# retriever_names = [
#     "Naive", "BM25", "Multi-Query", "Parent-Document", "Contextual Compression", "Ensemble"
# ]

results = []

import time

for sc in [False, True]:  # semantic chunking off/on
    for retriever_name in retriever_names: # loop through each retriever
    
        start_time = time.time()
        if sc:
            vectorstore = semantic_vectorstore
        else:
            vectorstore = text_vectorstore

        match retriever_name:
        
            case 'Naive':
                retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
                naive_retriever = retriever
            
            case 'BM25':
                retriever = BM25Retriever.from_documents(rag_documents) 
                bm25_retriever = retriever
            
            case 'Contextual Compression':
                compressor = CohereRerank(model="rerank-v3.5")
                retriever = ContextualCompressionRetriever(
                            base_compressor=compressor, base_retriever=naive_retriever)
                compression_retriever = retriever

            case 'Multi-Query':
                retriever = MultiQueryRetriever.from_llm(retriever=naive_retriever, llm=chat_model)
                multi_query_retriever = retriever
                
            case 'Parent-Document':
                parent_docs = rag_documents
                child_splitter = RecursiveCharacterTextSplitter(chunk_size=750)

                client = QdrantClient(location=":memory:")

                client.create_collection(
                    collection_name="full_documents",
                    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
                )

                parent_document_vectorstore = QdrantVectorStore(
                    collection_name="full_documents", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
                )
                store = InMemoryStore()
                retriever = ParentDocumentRetriever(
                                vectorstore = parent_document_vectorstore,
                                docstore=store,
                                child_splitter=child_splitter)
                retriever.add_documents(parent_docs, ids=None)
                parent_document_retriever = retriever

            case 'Ensemble':
                retriever_list = [naive_retriever, bm25_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
                equal_weighting = [1/len(retriever_list)] * len(retriever_list)
                retriever = EnsembleRetriever(retrievers=retriever_list, weights=equal_weighting)
            
            case ' ':
                retriever = "Invalid option"
        
        from operator import itemgetter
        from langchain_core.runnables import RunnablePassthrough, RunnableParallel
        from langchain.schema import StrOutputParser

        rag_chain = (
            {"context": itemgetter("question") | retriever, "question": itemgetter("question")} 
            | RunnablePassthrough.assign(context=itemgetter("context"))
            | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
        )

        feedback = rag_chain.invoke({"question" : "How does the Philippines regulate the development of AI in the country?"})
        feedback = feedback["response"].content

        # empathy version
        # rag_chain = (
        #         {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        #         | rag_prompt | chat_model | StrOutputParser()
        #     )

        # feedback = rag_chain.invoke({"question" : "How does the Philippines regulate the development of AI in the country?"})
        
        print(feedback)

        latency = time.time() - start_time
        
        evaluate(
            rag_chain.invoke,
            data=dataset_name,
            evaluators=[
                qa_evaluator,
                labeled_helpfulness_evaluator,
                empathy_evaluator],
            metadata={
                "retriever": retriever_name,
                "semantic chunking": sc,
                "empathy_prompt": empathy,
                "answer": feedback
                # "latency": latency
                }
            )

The Philippines regulates the development of AI through a framework that includes implementing a risk-based regulatory system, classifying AI systems by risk level, certifying and monitoring AI-related risks, and requiring registration of AI systems with the National Agency (NAIC). All entities involved in developing, deploying, or selling AI in the country must register their systems and comply fully with regulations. The regulation aims to promote responsible, ethical, and lawful use of AI, support Filipino innovation, and ensure AI contributes positively to society while minimizing harm.
View the evaluation results for experiment: 'virtual-line-5' at:
https://smith.langchain.com/o/92af5c49-0a9e-4f85-beea-085fbd240cd1/datasets/1917abde-886f-45e1-9091-14c439a6d1d3/compare?selectedSessions=6f6e746e-ff13-418f-822b-8970677dd111




0it [00:00, ?it/s]

KeyboardInterrupt: 

****** EVALUATION ENDS HERE ********
